# ==============================================
# 2_data_preprocessing.ipynb
# Bereinigung, Tokenisierung & Labeling (Feedback) für Feedback AI
# ==============================================

# Zelle 1: Importiere benötigte Bibliotheken
import os
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Für Jupyter Notebooks (falls benötigt)
%matplotlib inline

# Sicherstellen, dass NLTK-Daten vorhanden sind
nltk.download('punkt')

# Definiere Pfade zu den Rohdaten und zum Speicherort der verarbeiteten Daten
RAW_DATA_DIR = "../data/raw"
PROCESSED_DATA_DIR = "../data/processed"
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Zelle 2: Rohdaten laden
# Beispiel: Eine CSV-Datei "feedback_raw.csv", die die Spalten "generated_text" und "feedback" enthält.
# feedback: "thumbs_up" oder "thumbs_down"
raw_file = os.path.join(RAW_DATA_DIR, "feedback_raw.csv")
df = pd.read_csv(raw_file)
print(f"Anzahl der Rohdatensätze: {len(df)}")
display(df.head())

# Zelle 3: Datenbereinigung
def clean_text(text):
    """
    Entfernt HTML-Tags, überflüssige Leerzeichen und unerwünschte Zeichen.
    Konvertiert den Text in Kleinbuchstaben.
    """
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()

df["clean_text"] = df["generated_text"].apply(clean_text)
print("\nBereinigte Beispiele:")
display(df[["generated_text", "clean_text", "feedback"]].head())

# Zelle 4: Tokenisierung
def tokenize_text(text):
    """
    Zerlegt den Text in Tokens mithilfe von NLTK.
    """
    return word_tokenize(text)

df["tokens"] = df["clean_text"].apply(tokenize_text)
print("\nTokenisierte Beispiele:")
display(df[["clean_text", "tokens"]].head())

# Zelle 5: Vokabular erstellen
all_tokens = [token for tokens in df["tokens"] for token in tokens]
token_counts = Counter(all_tokens)
print(f"Anzahl eindeutiger Tokens: {len(token_counts)}")

# Begrenze das Vokabular optional auf die häufigsten Tokens
VOCAB_SIZE = 10000  # kann angepasst werden
most_common_tokens = token_counts.most_common(VOCAB_SIZE)

# Definiere Sondertokens
special_tokens = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
word2id = {}
idx = 0
for token in special_tokens:
    word2id[token] = idx
    idx += 1

for token, _ in most_common_tokens:
    if token not in word2id:
        word2id[token] = idx
        idx += 1

vocab_size = len(word2id)
print(f"Endgültige Vokabulargröße (inkl. Sondertokens): {vocab_size}")
id2word = {v: k for k, v in word2id.items()}

# Zelle 6: Tokens in IDs umwandeln
def tokens_to_ids(token_list, word2id, unk_id=word2id["<UNK>"]):
    return [word2id[t] if t in word2id else unk_id for t in token_list]

df["token_ids"] = df["tokens"].apply(lambda tokens: tokens_to_ids(tokens, word2id))
# Füge BOS und EOS zu jeder Token-Liste hinzu
BOS = word2id["<BOS>"]
EOS = word2id["<EOS>"]
df["token_ids"] = df["token_ids"].apply(lambda ids: [BOS] + ids + [EOS])
print("\nBeispiel für Token-IDs:")
display(df[["token_ids", "feedback"]].head())

# Zelle 7: Labeling des Feedbacks
# Hier weisen wir positive Beispiele (thumbs_up) einen höheren Gewichtungsfaktor zu,
# negative Beispiele (thumbs_down) einen geringeren. Die exakte Gewichtung kann experimentell angepasst werden.
def label_feedback(feedback):
    if feedback.strip().lower() == "thumbs_up":
        return 1  # oder einen höheren Wert, je nach gewünschtem Gewicht
    else:
        return 0  # oder einen niedrigeren Wert

df["feedback_label"] = df["feedback"].apply(label_feedback)
print("\nFeedback-Labels:")
display(df[["feedback", "feedback_label"]].head())

# Zelle 8: Train/Val/Test Split
# Wir teilen die Daten zufällig in 80 % Training und 20 % Test, und splitten dann noch in Validierung.
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Zelle 9: Speichern der verarbeiteten Daten und des Vokabulars
vocab_path = os.path.join(PROCESSED_DATA_DIR, "vocab.json")
with open(vocab_path, "w", encoding="utf-8") as f:
    json.dump(word2id, f, ensure_ascii=False)
print("Vokabular gespeichert unter:", vocab_path)

train_csv_path = os.path.join(PROCESSED_DATA_DIR, "train.csv")
val_csv_path = os.path.join(PROCESSED_DATA_DIR, "val.csv")
test_csv_path = os.path.join(PROCESSED_DATA_DIR, "test.csv")

train_df.to_csv(train_csv_path, index=False)
val_df.to_csv(val_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

print("\nVerarbeitete Daten gespeichert:")
print("Train CSV:", train_csv_path)
print("Val CSV:", val_csv_path)
print("Test CSV:", test_csv_path)

# Zelle 10: Fazit & Ausblick
print("""
Fazit:
- Die generierten Texte wurden erfolgreich bereinigt, tokenisiert und in numerische Sequenzen umgewandelt.
- Feedback wird in Form von Labels (1 für thumbs_up, 0 für thumbs_down) erfasst.
- Die Daten wurden in Trainings-, Validierungs- und Test-Sets aufgeteilt.
Nächste Schritte:
- Training eines Feedback-gesteuerten Modells, das lernt, bevorzugt Texte zu generieren, die positives Feedback erhalten.
- Analyse, ob sich bestimmte Textmuster mit positivem oder negativem Feedback korrelieren.
""")
